In [2]:
from eval_metrics import evaluate_metrics
from eval_metrics import evaluate_metrics_from_lists
from eval_metrics import combine_single_and_per_file_metrics

In [3]:
from util import get_file_list, get_padding, print_hparams, greedy_decode, \
    calculate_bleu, calculate_spider, LabelSmoothingLoss, beam_search, align_word_embedding, gen_str

/home/hj20/anaconda3/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [46]:
ref = 'Many different types of birds tweeting and whistling in the trees.'
pred='Many different types of birds are tweeting and singing'

In [4]:
from pathlib import Path
import json
import csv
from typing import Dict, List, Union, Tuple, Any

from coco_caption.pycocotools.coco import COCO
from coco_caption.pycocoevalcap.eval import COCOEvalCap

In [48]:
def write_json(data: Union[List[Dict[str, Any]], Dict[str, Any]],
               path: Path) \
        -> None:
    """ Write a dict or a list of dicts into a JSON file

    :param data: Data to write
    :type data: list[dict[str, any]] | dict[str, any]
    :param path: Path to the output file
    :type path: Path
    """
    with path.open("w") as f:
        json.dump(data, f)


In [39]:
tmp_dir = Path('tmp')

if not tmp_dir.is_dir():
    tmp_dir.mkdir()

ref_file = tmp_dir.joinpath('ref.json')
pred_file = tmp_dir.joinpath('pred.json')

write_json(ref, ref_file)
write_json(pred, pred_file)

NameError: name 'write_json' is not defined

In [50]:
ref_file 

PosixPath('tmp/ref.json')

In [40]:
tmp_dir = Path('tmp')
ref_file = tmp_dir.joinpath('ref.json')
pred_file = tmp_dir.joinpath('pred.json')

In [13]:
cocoEval.params['audio_id'][0]= cocoRes.getAudioIds()

In [14]:
cocoEval.evaluate()

TypeError: unhashable type: 'list'

In [22]:
 cocoEval.eval.items()

dict_items([('Bleu_1', 0.5150218257646578), ('Bleu_2', 0.34040158314669394), ('Bleu_3', 0.23015204254197477), ('Bleu_4', 0.15081783804225588), ('METEOR', 0.15907197362807568), ('ROUGE_L', 0.36777795785420464), ('CIDEr', 0.371045640092004), ('SPICE', 0.10943448773657152), ('SPIDEr', 0.24024006391428776)])

In [23]:
coco

In [24]:
cocoRes

In [27]:
cocoRes

In [41]:
coco = COCO(str(ref_file))
cocoRes = coco.loadRes(str(pred_file))

# Create evaluation object and evaluate metrics
cocoEval = COCOEvalCap(coco, cocoRes)
cocoEval.params['audio_id']= cocoRes.getAudioIds()
cocoEval.evaluate()

# Make dict from metrics
metrics = dict(
        (m, s) for m, s in cocoEval.eval.items()
    )

loading annotations into memory...
0:00:00.004715
creating index...
index created!
Loading and preparing results...     
DONE (t=0.00s)
creating index...
index created!
tokenization...
setting up scorers...
computing Bleu score...
{'testlen': 8060, 'reflen': 9433, 'guess': [8060, 7036, 6012, 4988], 'correct': [4922, 1877, 750, 251]}
ratio: 0.8544471536095776
Bleu_1: 0.515
Bleu_2: 0.340
Bleu_3: 0.230
Bleu_4: 0.151
computing METEOR score...
METEOR: 0.159
computing Rouge score...
ROUGE_L: 0.368
computing CIDEr score...
CIDEr: 0.371
computing SPICE score...
SPICE: 0.109
computing SPIDEr score...
SPIDEr: 0.240


In [ ]:
1028개 ref 1028*5
pred 1023

In [8]:
metrics

{'Bleu_1': 0.5150218257646578,
 'Bleu_2': 0.34040158314669394,
 'Bleu_3': 0.23015204254197477,
 'Bleu_4': 0.15081783804225588,
 'METEOR': 0.15907197362807568,
 'ROUGE_L': 0.36777795785420464,
 'CIDEr': 0.371045640092004,
 'SPICE': 0.10943448773657152,
 'SPIDEr': 0.24024006391428776}

In [ ]:
def calculate_spider(output_batch, ref_batch, word_dict_pickle_path):
    word_dict = get_word_dict(word_dict_pickle_path)
    word_dict[len(word_dict)] = '<pad>'
    special_token = ['<sos>', '<eos>', '<pad>']
    output_str = [ind_to_str(o, special_token, word_dict) for o in output_batch]
    ref_str = [[ind_to_str(r, special_token, word_dict) for r in ref] for ref in ref_batch]

    output_str = [' '.join(o) for o in output_str]
    ref_str = [[' '.join(r) for r in ref] for ref in ref_str]

    metrics, per_file_metrics = evaluate_metrics_from_lists(output_str, ref_str)
    score = metrics['SPIDEr']

    return score, output_str, ref_str

In [9]:
import torch
import torch.nn as nn
import time

from data_handling import get_clotho_loader, get_test_data_loader
from model import TransformerModel  # , RNNModel, RNNModelSmall
import itertools
import numpy as np
import os
import sys
import logging
import csv

from util import get_file_list, get_padding, print_hparams, greedy_decode, \
    calculate_bleu, calculate_spider, LabelSmoothingLoss, beam_search, align_word_embedding, gen_str
from hparams import hparams
from torch.utils.tensorboard import SummaryWriter

import argparse

hp = hparams()
parser = argparse.ArgumentParser(description='hparams for model')

device = torch.device(hp.device)
np.random.seed(hp.seed)
torch.manual_seed(hp.seed)


In [19]:
import numpy as np
import time
import torch
import torch.nn as nn

In [20]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.transformer import TransformerDecoder,TransformerDecoderLayer

from hparams import hparams as hp
from encoder import Cnn14,Cnn10,init_layer


In [21]:
def init_layer(layer):
    """Initialize a Linear or Convolutional layer. """
    nn.init.xavier_uniform_(layer.weight)

    if hasattr(layer, 'bias'):
        if layer.bias is not None:
            layer.bias.data.fill_(0.)
            
def init_bn(bn):
    """Initialize a Batchnorm layer. """
    bn.bias.data.fill_(0.)
    bn.weight.data.fill_(1.)

In [22]:
class Transfer_Cnn10(nn.Module):
    def __init__(self, freeze_base=True, pretrain_checkpoint=None):
        """Classifier for a new task using pretrained Cnn14 as a sub module.
        """
        super(Transfer_Cnn10, self).__init__()

        audioset_classes_num = 527
        self.base = Cnn10()

        #self.init_weights()

        if pretrain_checkpoint:
            self.load_from_pretrain(pretrain_checkpoint)
            
        #self.base.fc_audioset = nn.Linear(2048, classes_num, bias=True)
        # 안쓰이는이유는 multi-class clasification을 생략하기 때문,

        if freeze_base:
            # 2단계 freeze / 3단계 freeze X
            # Freeze AudioSet pretrained layers
            for param in self.base.parameters():
                param.requires_grad = False

    #def init_weights(self):
        #init_layer(self.fc_transfer)

    def load_from_pretrain(self, pretrained_checkpoint):
        pretrained_checkpoint="/home/hj20/dcase_2020_T6/Cnn10_mAP=0.380.pth"
        checkpoint = torch.load(pretrained_checkpoint)
        checkpoint['model'].pop('spectrogram_extractor.stft.conv_imag.weight')#가중치 삭제  
        checkpoint['model'].pop('spectrogram_extractor.stft.conv_real.weight')#가중치 삭제
        checkpoint['model'].pop('logmel_extractor.melW')#가중치 삭제
        checkpoint['model'].pop('fc1.weight')
        checkpoint['model'].pop('fc1.bias')
        checkpoint['model'].pop('fc_audioset.weight')
        checkpoint['model'].pop('fc_audioset.bias')
        self.base.load_state_dict(checkpoint['model'])

    def forward(self, input):
        """Input: (batch_size, data_length)
        """
        output = self.base(input)

        #embedding = output_dict['embedding']
        #clipwise_output = output_dict['clipwise_output']

        return output #, clipwise_output
 

In [23]:
pretrained_checkpoint="/home/hj20/dcase_2020_T6/Cnn10_mAP=0.380.pth"


In [24]:
cnn=Transfer_Cnn10(freeze_base=True,pretrain_checkpoint="/home/hj20/dcase_2020_T6/Cnn10_mAP=0.380.pth")

In [25]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.transformer import TransformerDecoder,TransformerDecoderLayer

from hparams import hparams as hp
from encoder import Cnn14,Transfer_Cnn14,init_layer


class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=100):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, batch_size, dropout=0.5,pretrain_cnn=None,
                 pretrain_emb=None,freeze_cnn=True):
        super(TransformerModel, self).__init__()

        self.model_type = 'cnn10+transformer'
        decoder_layers = TransformerDecoderLayer(d_model=nhid, nhead=nhead, dropout=dropout)
        self.transformer_decoder = TransformerDecoder(decoder_layers, nlayers)
        self.word_emb = nn.Embedding(ntoken, nhid)
        self.ninp = ninp
        self.nhid = nhid
        self.fc = nn.Linear(512, 512, bias=True)
        self.fc1 = nn.Linear(512, nhid, bias=True)
        self.dec_fc = nn.Linear(nhid, ntoken)
        self.batch_size = batch_size
        self.ntoken = ntoken

        #def __init__(self, freeze_base, pretrain_checkpoint=None):
        pretrain_cnn="/home/hj20/dcase_2020_T6/Cnn10_mAP=0.380.pth"
        
        #self.encoder = Transfer_ResNet54(freeze_base=freeze_cnn, pretrain_checkpoint=pretrain_cnn)
        self.encoder = cnn
        self.dropout = nn.Dropout(dropout)
        self.pos_encoder = PositionalEncoding(nhid, dropout)
        self.generator = nn.Softmax(dim=-1)
        self.init_weights()

        '''
        if pretrain_cnn is not None:
            dict_trained = pretrain_cnn
            dict_new = self.encoder.state_dict().copy()
            new_list = list(self.encoder.state_dict().keys())
            trained_list = list(dict_trained.keys())
            for i in range(len(new_list)):
                dict_new[new_list[i]] = dict_trained[trained_list[i]]
            self.encoder.load_state_dict(dict_new)
        
        if freeze_cnn:
            self.freeze_cnn()
        '''

        if pretrain_emb is not None:
            self.word_emb.weight.data = pretrain_emb

    '''
    def freeze_cnn(self):
        for p in self.encoder.parameters():
            p.requires_grad = False
    '''

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        init_layer(self.fc1)
        init_layer(self.fc)
        self.word_emb.weight.data.uniform_(-initrange, initrange)
        self.dec_fc.bias.data.zero_()
        self.dec_fc.weight.data.uniform_(-initrange, initrange)

    def encode(self, src, input_mask=None):
        global x 
        x = self.encoder(src)  # (batch_size, 2048, T/16, mel_bins/16) ,mixup
        x = torch.mean(x, dim=3)  # (batch_size, 2048, T/16)
        x = x.permute(2, 0, 1)  # (T/16,batch_size,2048)
        x = F.relu_(self.fc(x))
        x = F.dropout(x, p=0.2, training=self.training)
        x = torch.relu(self.fc1(x))
        return x

    def decode(self, mem, tgt, input_mask=None, target_mask=None, target_padding_mask=None):
        # tgt:(batch_size,T_out)
        # mem:(T_mem,batch_size,nhid)

        tgt = tgt.transpose(0, 1)  # (T_out,batch_size)
        if target_mask is None or target_mask.size(0) != len(tgt):
            device = tgt.device
            target_mask = self.generate_square_subsequent_mask(len(tgt)).to(device)

        tgt = self.dropout(self.word_emb(tgt)) * math.sqrt(self.nhid)
        tgt = self.pos_encoder(tgt)
        # mem = self.pos_encoder(mem)
        output = self.transformer_decoder(tgt, mem, memory_mask=input_mask, tgt_mask=target_mask,
                                          tgt_key_padding_mask=target_padding_mask)
        output = self.dec_fc(output)
        return output

    def forward(self, src, tgt, input_mask=None, target_mask=None, target_padding_mask=None):
        # src:(batch_size,T_in,feature_dim)
        # tgt:(batch_size,T_out)
        mem = self.encode(src)
        output = self.decode(mem, tgt, input_mask=input_mask, target_mask=target_mask,
                             target_padding_mask=target_padding_mask)
        return output


In [27]:
pretrain_emb = align_word_embedding(hp.word_dict_pickle_path, hp.pretrain_emb_path, hp.ntoken,
                                        hp.nhid) if hp.load_pretrain_emb else None

In [28]:
model = TransformerModel(hp.ntoken, hp.ninp, hp.nhead, hp.nhid, hp.nlayers, hp.batch_size, dropout=0.2,
                             pretrain_cnn="/home/hj20/dcase_2020_T6/Cnn10_mAP=0.380.pth", pretrain_emb=pretrain_emb, freeze_cnn=True).to(device)

In [11]:
evaluation_beam = get_clotho_loader(data_dir=hp.data_dir, split='evaluation',
                                        input_field_name='features',
                                        output_field_name='words_ind',
                                        load_into_memory=False,
                                        batch_size=32,
                                        nb_t_steps_pad='max',
                                        shuffle=False,
                                        return_reference=True)

In [16]:
src, tgt, tgt_len, ref = next(iter(evaluation_beam))

In [18]:
src = src.to(device)


In [33]:
src[0]

tensor([[  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
        [  0.0000,   0.0000,   0.0000,  ...,   0.0000,   0.0000,   0.0000],
        ...,
        [-12.5787, -10.3935,  -8.6105,  ..., -41.5464, -42.8459, -43.2396],
        [-10.6613,  -8.3408,  -7.2884,  ..., -41.8352, -42.3398, -43.2158],
        [ -9.5568, -10.8398,  -9.9628,  ..., -42.3450, -44.2460, -45.3972]],
       device='cuda:0')

In [34]:
output[0]

tensor([   0, 2501,  667,  421, 3916, 2121,  891,  699, 1523, 2648, 2564,  942,
        1078, 2503, 3305, 1823, 1584, 1391, 2200, 1823, 1956, 3440,  699, 1929,
        2409, 4270, 3354, 3857,  906, 4254,  421, 4106, 3511, 3421, 3338,  359,
        3459, 4149, 4027, 1823,  563, 2220, 3142, 3713,  414, 2869, 3338, 3089,
         112, 1700, 1623,  540, 1945, 3857, 3270, 1423,  732, 2874,  137, 3037,
        1721,  979, 3528, 1462, 2893, 1312, 3671,  719, 4172,  801, 3033,  699,
         936, 1312, 1312, 4172, 1923, 4159, 1683, 2884, 2648,  606, 1116, 3270,
        2028, 2662,  152, 4069,  519, 1265, 2718, 4220, 2274, 4218, 3997, 3000,
        1464, 2718, 2263,  241], device='cuda:0')

In [36]:
output =beam_search(model, src, max_len=30, start_symbol_ind=0, beam_size=3)

In [41]:
output[0]

tensor([   0, 1685, 3917, 3865, 2462, 2102,  316, 2102, 4272, 1823, 2115,  426,
        1146, 3494, 1719, 3933, 4239, 3917, 1136, 1386,  766, 1767, 3440,  699,
        3440,  437, 3354, 2776, 1312, 4194,   47], device='cuda:0')

In [ ]:

            output_sentence_ind_batch = []
            for i in range(output.size()[0]):
                output_sentence_ind = []
                for j in range(1, output.size(1)):
                    sym = output[i, j]
                    if sym == eos_ind: break
                    output_sentence_ind.append(sym.item())
                output_sentence_ind_batch.append(output_sentence_ind)
            output_sentence_all.extend(output_sentence_ind_batch)
            ref_all.extend(ref)
        score, output_str, ref_str = calculate_spider(output_sentence_all, ref_all, word_dict_pickle_path)


In [6]:
output_str='Many different types of birds are tweeting and singing'

In [7]:
metrics, per_file_metrics = evaluate_metrics_from_lists(output_str, ref_str)

AssertionError: 

In [ ]:
score, output_str, ref_str = calculate_spider(output_sentence_all, ref_all, word_dict_pickle_path)